In [ ]:
# Input
outputIndex = 0
outputFile = 'meetupType/prototypeSentences_' + outputIndex + '.csv'

In [12]:
# Prepare seed terms for each meetup type

meetup_types = {"Music making" : [
"Conduct",
"Perform",
"Join",
"Produce",
"Write",
"Sing",
"Play",
# "Song",
"Record"],
"Business meeting": [
"Sign",
"Agree",
"Contract",
"Commit",
"Retire",
"Career",
"Hire",
"Debut"],
"Personal life": [
"Born",
"Die",
"Marry",
"Divorce",
"Adult",
"Child",
"Elder",
"Young",
"Father",
"Mother",
"Family"],
"Coincidence": [
"Meet",
"Stumble",
"Encounter",
"Find",
"Discover",
"Trigger"],
"Public celebration": [
"Award",
"Ceremony",
"Marriage",
"Funeral",
"Public",
"Celebration"],
"Education": [
"Learn",
"Teach",
"Mentor",
"Degree",
"University",
"Academia",
"Conservatoire"]}

print(meetup_types)



{'Music making': ['Conduct', 'Perform', 'Join', 'Produce', 'Write', 'Sing', 'Play', 'Song', 'Record'], 'Business meeting': ['Sign', 'Agree', 'Contract', 'Commit', 'Retire', 'Career', 'Hire', 'Debut'], 'Personal life': ['Born', 'Die', 'Marry', 'Divorce', 'Adult', 'Child', 'Elder', 'Young', 'Father', 'Mother', 'Family'], 'Coincidence': ['Meet', 'Stumble', 'Encounter', 'Find', 'Discover', 'Trigger'], 'Public celebration': ['Award', 'Ceremony', 'Marriage', 'Funeral', 'Public', 'Celebration'], 'Education': ['Learn', 'Teach', 'Mentor', 'Degree', 'University', 'Academia', 'Conservatoire']}


In [2]:
# Gather prototype sentences and assign them to types
#### Sentence -> lemmas -> match any seed
# Compare query sentences with seed via sentence embeddings



In [15]:
folder = "indexedSentences"
import glob
files = set(glob.glob(folder + "/*"))
print(len(files), " files" )
import random
samples = random.sample(files, 100)
print(100, " samples" )
print(samples)

1002  files
100  samples
['indexedSentences/155965.csv', 'indexedSentences/2553132.csv', 'indexedSentences/1103482.csv', 'indexedSentences/1882145.csv', 'indexedSentences/493715.csv', 'indexedSentences/3614145.csv', 'indexedSentences/1186480.csv', 'indexedSentences/312781.csv', 'indexedSentences/45968.csv', 'indexedSentences/1310858.csv', 'indexedSentences/1422240.csv', 'indexedSentences/3606301.csv', 'indexedSentences/238175.csv', 'indexedSentences/1253783.csv', 'indexedSentences/3488554.csv', 'indexedSentences/3372135.csv', 'indexedSentences/2672499.csv', 'indexedSentences/3190132.csv', 'indexedSentences/1253793.csv', 'indexedSentences/966302.csv', 'indexedSentences/577944.csv', 'indexedSentences/2512661.csv', 'indexedSentences/523339.csv', 'indexedSentences/45450.csv', 'indexedSentences/1277042.csv', 'indexedSentences/589693.csv', 'indexedSentences/2508330.csv', 'indexedSentences/2508150.csv', 'indexedSentences/3227606.csv', 'indexedSentences/322567.csv', 'indexedSentences/3535198.c

In [16]:
# Look for sentences matching any seed
# Using Spacy
# Install spaCy (run in terminal/prompt)
#import sys
#!{sys.executable} -m pip install spacy
#
# Download spaCy's  'en' Model
#!{sys.executable} -m spacy download en

#### Sentence -> lemmas -> match any seed
import spacy
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
sentence = "The striped bats are hanging on their feet for best"
doc = nlp(sentence)
print("hang" in [token.lemma_ for token in doc])
#
# For each meetup type, load sentences matching any of the seed terms, in order to get a dictionary of types and prototype sentences.
# nlp in scope

True


In [17]:
# nlp is in scope

# Clean seeds
for mtype, mseeds in meetup_types.items():
    doc = nlp(" ".join(mseeds).lower())
    meetup_types[mtype] = [token.lemma_ for token in doc]

print(meetup_types)
for mtype, mseeds in meetup_types.items():
    print(mtype, type(mseeds))

{'Music making': ['conduct', 'perform', 'join', 'produce', 'write', 'sing', 'play', 'song', 'record'], 'Business meeting': ['sign', 'agree', 'contract', 'commit', 'retire', 'career', 'hire', 'debut'], 'Personal life': ['bear', 'die', 'marry', 'divorce', 'adult', 'child', 'elder', 'young', 'father', 'mother', 'family'], 'Coincidence': ['meet', 'stumble', 'encounter', 'find', 'discover', 'trigger'], 'Public celebration': ['award', 'ceremony', 'marriage', 'funeral', 'public', 'celebration'], 'Education': ['learn', 'teach', 'mentor', 'degree', 'university', 'academia', 'conservatoire']}
Music making <class 'list'>
Business meeting <class 'list'>
Personal life <class 'list'>
Coincidence <class 'list'>
Public celebration <class 'list'>
Education <class 'list'>


In [31]:
import csv

#sample_file = samples[0]

prototypeSentences = []
for sample_file in samples:
    with open(sample_file) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        for row in csv_reader:
            if line_count == 0:
                #print(f'Column names are {", ".join(row)}')
                line_count += 1
            else:
                line_count += 1
                #
                text = row[0]
                sentence = row[1]
                paragraph = row[2]
                section = row[3]
                wikiId = row[4]
                doc = nlp(text)
                terms = [ token.lemma_ for token in doc ]
                candidateTypes = {}
                include = False
                for mtype, mseeds in meetup_types.items():
                    matchSeed = set(mseeds).intersection(set(terms))
                    candidateTypes[mtype] = len(matchSeed) > 0
                    if candidateTypes[mtype]:
                        include = True
                data = {}
                if include:
                    data["sentences"] = text
                    data["sentenceIndex"] = sentence
                    data["paragraphIndex"] = paragraph
                    data["section"] = section
                    data["file"] = sample_file
                    data["wikiId"] = wikiId
                    for mtype, match in candidateTypes.items():
                        data[mtype] = match
                    #print(data)
                    prototypeSentences.append(data)
print( len(prototypeSentences) )

2651


In [36]:
# Write prototypes to file
columns = prototypeSentences[0].keys()

with open(outputFile, 'w', newline='') as output_file:
    dict_writer = csv.DictWriter(output_file,  fieldnames=columns)
    dict_writer.writeheader()
    dict_writer.writerows(prototypeSentences)

In [35]:
# print(prototypeSentences[1])
# print(prototypeSentences[2])
# print(prototypeSentences[3])

{'sentences': 'His band included trumpeters Dizzy Gillespie, Jonah Jones, and Adolphus "Doc" Cheatham, saxophonists Ben Webster and Leon "Chu" Berry, guitarist Danny Barker, bassist Milt Hinton, and drummer Cozy Cole.Calloway had several hit records in the 1930s and 1940s, becoming known as the "Hi-de-ho" man of jazz for his most famous song, "Minnie the Moocher", originally recorded in 1931.', 'sentenceIndex': '3', 'paragraphIndex': '0', 'section': 'N/A', 'file': 'indexedSentences/155965.csv', 'wikiId': '155965', 'Music making': True, 'Business meeting': False, 'Personal life': False, 'Coincidence': False, 'Public celebration': False, 'Education': False}
{'sentences': 'His career saw renewed interest when he appeared in the 1980 film The Blues Brothers.', 'sentenceIndex': '8', 'paragraphIndex': '0', 'section': 'N/A', 'file': 'indexedSentences/155965.csv', 'wikiId': '155965', 'Music making': False, 'Business meeting': True, 'Personal life': False, 'Coincidence': False, 'Public celebrat

In [ ]:
# Using the seed sentences as training set with a sentencebert based classifier
# https://medium.com/dair-ai/tl-dr-sentencebert-8dec326daf4e
# Models https://www.sbert.net/docs/pretrained_models.html

from sentence_transformers import SentenceTransformer 

model = SentenceTransformer('all-mpnet-base-v2')
#Sentences we want to encode. Example: 
sentence = ['This framework generates embeddings for each input sentence']
 
#Sentences are encoded by calling model.encode() 
embedding = model.encode(sentence)
